<h1>Overview<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Load the Data</a></span></li><li><span><a href="#Create-Model" data-toc-modified-id="Create-Model-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Create Model</a></span></li><li><span><a href="#Compile-Model" data-toc-modified-id="Compile-Model-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Compile Model</a></span></li><li><span><a href="#Fit-Model" data-toc-modified-id="Fit-Model-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Fit Model</a></span></li></ul></li><li><span><a href="#Terminal" data-toc-modified-id="Terminal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Terminal</a></span></li></ul></div>

In [81]:
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda
from keras.callbacks import Callback
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention

from model import CompressiveTransformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the Data

In [2]:
train_data = pd.read_pickle('../data/processed/spooky-author/train.pkl')

x_train = np.array(train_data.x.tolist())
y_train = np.array(train_data.y.tolist())

_x_train = np.zeros((x_train.shape[0], 128))  # samples, d_model
_x_train[:,:x_train.shape[1]] = x_train
x_train = _x_train

## Create Model

In [148]:
d_model = 256
sequence_length = 128

model = CompressiveTransformer(d_model=d_model, sequence_length=sequence_length, batch_size=1)

tracking <tf.Variable 'memory:0' shape=(1, 512, 256) dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)> memory
tracking <tf.Variable 'compressed_memory:0' shape=(1, 512, 256) dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)> compressed_memory
Tensor("concatenate_52/concat:0", shape=(None, 1024, 256), dtype=float32)
Tensor("embedding_55/embedding_lookup/Identity_1:0", shape=(None, 128, 256), dtype=float32)
Tensor("concatenate_52/concat:0", shape=(None, 1024, 256), dtype=float32)
Tensor("concatenate_52/concat:0", shape=(Non

In [149]:
model.summary()

Model: "CompressiveTransformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 128)          0                                            
__________________________________________________________________________________________________
memory (InputLayer)             (None, 512, 256)     0                                            
__________________________________________________________________________________________________
compressed_memory (InputLayer)  (None, 512, 256)     0                                            
__________________________________________________________________________________________________
embedding_55 (Embedding)        (None, 128, 256)     5120000     x[0][0]                          
_____________________________________________________________________________

## Compile Model

In [150]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy')

## Fit Model

In [151]:
x_train.shape

(19579, 128)

In [152]:
for batch in range(0, len(x_train), 1):
    x_batch = x_train[batch:batch+1, :]
    y_batch = y_train[batch:batch+1]
    
    model.train_on_batch(x_batch,
                         y_batch)
    break


# model.fit(x_train, 
#           y_train,
#           epochs=8,
#           batch_size=32,
#           validation_split=0.3)

ValueError: Error when checking target: expected mlp_layer_norm_L0 to have 3 dimensions, but got array with shape (1, 3)

# Terminal

In [24]:
x = K.variable([1,2,3])

In [63]:
a = np.array([i for i in range(200)]).reshape((10, 20))
b = np.array([i*100 for i in range(100)]).reshape((5, 20))

a = K.variable(a)
b = K.variable(b)

In [64]:
# K.concatenate([a,b], axis=0)

In [84]:
a = K.zeros((2, 5, 20))
b = K.zeros((2, 10, 20))

c = K.concatenate([a, b], axis=1)
print(c.shape)

(2, 15, 20)


In [87]:
a[:, 5:, :]

<tf.Tensor: shape=(2, 0, 20), dtype=float32, numpy=array([], shape=(2, 0, 20), dtype=float32)>

In [105]:
a = K.variable(np.array([i for i in range(200)]).reshape((2, 5, 20)))

def call(x, units=None, gain=None, bias=None):
    if units is None:
        units = np.prod(x.shape[1:])
        print(f'units={units}')
    
    mean = K.sum(x) / units
    std_dev = K.sqrt(K.sum(K.square(x - mean)) / units)

    y = (x - mean) / std_dev
    if gain:
        y *= gain
    if bias:
        y += bias
    return y

K.eval(call(a))

units=100


array([[[-1.2232091 , -1.2170624 , -1.2109156 , -1.2047688 ,
         -1.1986221 , -1.1924753 , -1.1863285 , -1.1801817 ,
         -1.174035  , -1.1678882 , -1.1617414 , -1.1555946 ,
         -1.1494478 , -1.143301  , -1.1371542 , -1.1310074 ,
         -1.1248606 , -1.1187139 , -1.1125672 , -1.1064204 ],
        [-1.1002736 , -1.0941268 , -1.08798   , -1.0818332 ,
         -1.0756865 , -1.0695397 , -1.0633929 , -1.0572461 ,
         -1.0510993 , -1.0449525 , -1.0388057 , -1.032659  ,
         -1.0265123 , -1.0203655 , -1.0142187 , -1.0080719 ,
         -1.0019251 , -0.9957783 , -0.98963153, -0.98348475],
        [-0.97733796, -0.9711912 , -0.96504444, -0.95889765,
         -0.95275086, -0.9466041 , -0.9404573 , -0.9343105 ,
         -0.92816377, -0.922017  , -0.9158702 , -0.9097234 ,
         -0.9035766 , -0.8974298 , -0.89128304, -0.8851363 ,
         -0.8789895 , -0.8728427 , -0.86669594, -0.86054915],
        [-0.85440236, -0.84825563, -0.84210885, -0.83596206,
         -0.82981527,